# 설치
  - `pip install langchain langchain-qdrant langchain-openai python-dotenv`

In [1]:
!uv pip install langchain-qdrant

Resolved 35 packages in 367ms
Prepared 1 package in 47ms
Installed 1 package in 35ms
 + langchain-qdrant==1.1.0


# Vector Store 생성
- QDrantClient로 Database에 연결
    - Collection 생성(없으면)
- VectorSore 생성
    - client와 collection name, embedding 모델을 전달해서 생성

In [ ]:
from uuid import uuid4
uuid4() # 중복될 값이 거의 절대 안나오도록하는 알고리즘. - ID(식별자) 만들 때 사용.

UUID('7d0e23ea-53e4-4576-9c19-350c6362e1c2')

In [6]:
from uuid import uuid4
from langchain_core.documents import Document
data = [
    {
        "name": "The Time Machine",
        "description": "시간 여행자는 미래로 여행을 떠나 인류의 진화와 몰락을 목격하게 된다. 그는 멀리 떨어진 미래 사회에서 인간 문명이 퇴화한 모습을 보고 충격을 받는다. 이 이야기는 인간 문명과 과학기술의 진보가 가져올 결과를 철학적으로 탐구한다.",
        "author": "H.G. Wells",
        "year": 1895
    },
    {
        "name": "Ender's Game",
        "description": "소년 엔더는 외계 종족 '버거'와의 전쟁을 대비해 혹독한 군사 훈련을 받는다. 그는 전략적 천재성을 발휘하여 점차 전쟁의 중심에 서게 된다. 이야기는 어린아이를 도구로 삼는 전쟁의 비인간성과 심리적 고통을 그린다.",
        "author": "Orson Scott Card",
        "year": 1985
    },
    {
        "name": "Brave New World",
        "description": "유전자 조작과 세뇌로 계급이 철저히 나뉘어진 미래 사회가 배경이다. 인간은 자유 없이 쾌락과 안정을 주입받으며 살고, 비순응자는 배제된다. 디스토피아 속에서 인간 본성과 자유의 의미를 묻는 소설이다.",
        "author": "Aldous Huxley",
        "year": 1932
    },
    {
        "name": "The Hitchhiker's Guide to the Galaxy",
        "description": "지구 멸망 직후 우주로 탈출한 주인공이 외계 친구와 함께 기상천외한 모험을 시작한다. 우주의 황당한 존재들과 철학적 농담들이 넘치는 코믹 SF다. 우주에 대한 진지한 질문을 유쾌하게 풀어낸 작품이다.",
        "author": "Douglas Adams",
        "year": 1979
    },
    {
        "name": "Dune",
        "description": "사막 행성 아라키스를 둘러싼 정치적 음모와 권력 투쟁이 벌어진다. 주인공 폴은 운명과 예언에 따라 거대한 반란을 이끌게 된다. 정치, 종교, 생태학이 얽힌 거대한 서사의 서막이다.",
        "author": "Frank Herbert",
        "year": 1965
    },
    {
        "name": "Foundation",
        "description": "수학자 하리 셀던은 미래를 예측하는 '심리역사학'을 개발해 인류의 몰락을 막기 위한 계획을 세운다. 그는 제국의 붕괴 속에서도 문명을 재건할 수 있는 재단을 설립한다. 과학과 지식의 힘이 문명을 구원할 수 있는지를 탐구한다.",
        "author": "Isaac Asimov",
        "year": 1951
    },
    {
        "name": "Snow Crash",
        "description": "가상현실 '메타버스'와 현실이 뒤얽힌 미래에서, 주인공 해커는 '스노 크래시'라는 치명적인 바이러스를 추적한다. 정보와 언어, 권력이 하나로 연결된 세계가 그려진다. 포스트사이버펑크의 대표작으로 인터넷 이후의 세계를 예견한다.",
        "author": "Neal Stephenson",
        "year": 1992
    },
    {
        "name": "Neuromancer",
        "description": "해커 '케이스'는 거대 기업의 의뢰를 받아 사이버 공간에서 불가능한 해킹을 수행한다. 임무를 수행하며 인공지능과 인간의 경계가 모호해지는 음모에 휘말린다. 사이버펑크 장르의 원형으로 가상 현실과 인간 의식의 융합을 다룬다.",
        "author": "William Gibson",
        "year": 1984
    },
    {
        "name": "The War of the Worlds",
        "description": "화성인이 지구를 침략하면서 인류는 공포와 절망 속에 빠진다. 압도적인 기술력 앞에 속수무책으로 무너지는 인간의 모습이 그려진다. 외계 생명체와 문명의 충돌을 통해 인간 중심적 세계관에 경종을 울린다.",
        "author": "H.G. Wells",
        "year": 1898
    },
    {
        "name": "The Hunger Games",
        "description": "디스토피아 사회에서 어린이들은 생존 게임에 강제로 참가해야 한다. 주인공 캣니스는 잔혹한 게임 속에서 인간성과 저항의 상징이 되어간다. 권력과 억압, 생존 본능의 이야기가 펼쳐진다.",
        "author": "Suzanne Collins",
        "year": 2008
    },
    {
        "name": "The Andromeda Strain",
        "description": "우주에서 온 미생물이 미국의 한 마을을 초토화시키자, 과학자들이 이를 분석하고 대응에 나선다. 시간과의 싸움 속에서 인류의 과학 기술과 오만함이 드러난다. 생물무기와 전염병의 공포를 현실감 있게 묘사한 작품이다.",
        "author": "Michael Crichton",
        "year": 1969
    },
    {
        "name": "The Left Hand of Darkness",
        "description": "외교관 제너리 아이가 젠더가 유동적인 외계 종족이 사는 행성에 파견된다. 그는 그들과의 문화적 충돌과 이해를 통해 진정한 인간성과 연대를 배워간다. 젠더와 정체성, 타자성에 대한 철학적 탐구가 돋보이는 작품이다.",
        "author": "Ursula K. Le Guin",
        "year": 1969
    },
    {
        "name": "The Three-Body Problem",
        "description": "문화대혁명기의 중국에서 외계 문명과의 접촉이 시작되며 인류는 큰 위기에 직면한다. 세 개의 태양이 있는 불안정한 행성에 사는 외계인과의 접촉이 인류 문명에 위협을 가한다. 과학, 철학, 문명 충돌을 고찰하는 하드 SF 걸작이다.",
        "author": "Liu Cixin",
        "year": 2008
    }
]

documents = [
    Document(page_content=doc['description'], metadata=doc, id=str(uuid4())) for doc in data
]
documents

[Document(id='18bbabf7-3176-484e-b9e1-29af968df565', metadata={'name': 'The Time Machine', 'description': '시간 여행자는 미래로 여행을 떠나 인류의 진화와 몰락을 목격하게 된다. 그는 멀리 떨어진 미래 사회에서 인간 문명이 퇴화한 모습을 보고 충격을 받는다. 이 이야기는 인간 문명과 과학기술의 진보가 가져올 결과를 철학적으로 탐구한다.', 'author': 'H.G. Wells', 'year': 1895}, page_content='시간 여행자는 미래로 여행을 떠나 인류의 진화와 몰락을 목격하게 된다. 그는 멀리 떨어진 미래 사회에서 인간 문명이 퇴화한 모습을 보고 충격을 받는다. 이 이야기는 인간 문명과 과학기술의 진보가 가져올 결과를 철학적으로 탐구한다.'),
 Document(id='774c53ac-fa42-46f9-985f-38f22ad77a41', metadata={'name': "Ender's Game", 'description': "소년 엔더는 외계 종족 '버거'와의 전쟁을 대비해 혹독한 군사 훈련을 받는다. 그는 전략적 천재성을 발휘하여 점차 전쟁의 중심에 서게 된다. 이야기는 어린아이를 도구로 삼는 전쟁의 비인간성과 심리적 고통을 그린다.", 'author': 'Orson Scott Card', 'year': 1985}, page_content="소년 엔더는 외계 종족 '버거'와의 전쟁을 대비해 혹독한 군사 훈련을 받는다. 그는 전략적 천재성을 발휘하여 점차 전쟁의 중심에 서게 된다. 이야기는 어린아이를 도구로 삼는 전쟁의 비인간성과 심리적 고통을 그린다."),
 Document(id='60409d7c-ef94-4890-94c7-fa5a99f4a016', metadata={'name': 'Brave New World', 'description': '유전자 조작과 세뇌로 계급이 철저히 나뉘어진 미래 사회가 배경이다. 인간은 자유 없이 쾌락과 안정을 주입받으며 살고

In [ ]:
# doc = {
#         "name": "The Three-Body Problem",
#         "description": "문화대혁명기의 중국에서 외계 문명과의 접촉이 시작되며 인류는 큰 위기에 직면한다. 세 개의 태양이 있는 불안정한 행성에 사는 외계인과의 접촉이 인류 문명에 위협을 가한다. 과학, 철학, 문명 충돌을 고찰하는 하드 SF 걸작이다.",
#         "author": "Liu Cixin",
#         "year": 2008
#     }
# {key:value for key, value in doc.items() if key != "description"}

{'name': 'The Three-Body Problem', 'author': 'Liu Cixin', 'year': 2008}

In [ ]:
# qdrant db 실행 - docker desktop 먼저 실행해야함.
# docker run -p 6333:6334 -v "qdrant_storage:/qdrant/storage:z" qdrant/qdrant
docker run -p 6333:6333 -p 6334:6334 -v "qdrant_storage:/qdrant/storage:z"  qdrant/qdrant

In [2]:
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings

from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# Embedding 모델을 성능 좋은 것을 쓰면 된다.(LLM모델과 연동될 필요 없다.)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

COLLECTION_NAME = "sf_books"
VECTOR_SIZE = 1536 # text-embedding-3-small의 Embedding vector 차원. (임베딩 모델의 차원수에 맞춰준다.)

# Qdrant 연결
client = QdrantClient(host="localhost", port=6333)

# sf_books collection이 있으면 삭제
if client.collection_exists(COLLECTION_NAME):
    client.delete_collection(COLLECTION_NAME)

# Collection 생성
client.create_collection(
    collection_name=COLLECTION_NAME,
    # 벡터 저장소에 대한 설정.
    vectors_config=VectorParams(
        size=VECTOR_SIZE,
        distance=Distance.COSINE
    )
)

True

In [4]:
# QdrantVectorStore 생성 -> Langchain의 VectorStore: Langchain이 제공하는 동일한
# interface로 VectorDB와 CRUD를 할 수 있게 해주는 Runnable 타입(LCEL에 추가 가능)

# 단순 연결: QdrantVectorStore() 객체를 생성
# 데이터를 넣으면서 생성: QdrantVectorStore.from_documents()

# 단순 연결 VectorStore
vectorstore = QdrantVectorStore(
    client=client, # QdrantClient
    embedding=embeddings, # Embedding Model
    collection_name=COLLECTION_NAME # 연결할 Collection 지정.
)
# 데이터 추가(upsert) - add_documents([Document])
ids = vectorstore.add_documents(documents)
print(ids) # upsert된 point들의 id들을 반환.
# localhost:6333/dashboard

['31c12211-5bbd-497b-8492-69d9e8b544fb', '5a88c7d7-3c84-4f1b-b01e-d80b57dfb31b', '8e380646-6592-4908-b254-1e427f221dce', '2034a84b-51a4-4d68-8179-2064ea53d0fa', '098ac280-f388-4389-85bd-0885bb4f9300', '794599a1-2410-4218-8b9b-a095bb1314d0', '35a36245-48e0-4da7-baf7-d9985f60a808', '42840547-c3e6-4c18-a6ec-d205b86bebac', '8ec94538-2a2f-4e06-95f8-fee8a85d0a82', 'aab54076-9df3-4048-8340-363cdc6998a4', 'c96e5ec6-4705-4db0-abcc-83aa34787f5c', '304fe001-69cb-4dd9-b380-e02033981f53', '09a23a5e-36a6-4207-8495-f3dd44190faa']


In [5]:
##################################
# Vector Store로부터 정보조회

print(vectorstore.collection_name)
print(vectorstore.distance)
print(vectorstore.content_payload_key) # page_content가 payload 어떤 key에 매핑됐는지
print(vectorstore.metadata_payload_key) # metadata가 payload 어떤 key에 매핑됐는지
print(vectorstore.client) # QdrantClient 객체를 조회.

sf_books
Cosine
page_content
metadata


In [6]:
from qdrant_client.models import PayloadSchemaType
# payload index 생성
client = vectorstore.client

client.create_payload_index(
    collection_name=COLLECTION_NAME,
    field_name="year",
    field_schema=PayloadSchemaType.INTEGER # DATETIME - 년-월-일-시-분-초 날짜정보.
)

client.create_payload_index(
    collection_name=COLLECTION_NAME,
    field_name="name",
    field_schema=PayloadSchemaType.TEXT # 문자열-TEXT : 부분 일치 조회.    
)

client.create_payload_index(
    collection_name=COLLECTION_NAME,
    field_name="author", # 저자
    field_schema=PayloadSchemaType.KEYWORD, # 문자열-KEYWORD : 완전 일치.    
)

UpdateResult(operation_id=7, status=<UpdateStatus.COMPLETED: 'completed'>)

In [7]:
# Collection 정보확인
collection_info = client.get_collection(COLLECTION_NAME)
print("Point(데이터)개수:", collection_info.points_count)
print("Payload Index조회")
collection_info.payload_schema

Point(데이터)개수: 13
Payload Index조회


{'author': PayloadIndexInfo(data_type=<PayloadSchemaType.KEYWORD: 'keyword'>, params=None, points=0),
 'name': PayloadIndexInfo(data_type=<PayloadSchemaType.TEXT: 'text'>, params=None, points=0),
 'year': PayloadIndexInfo(data_type=<PayloadSchemaType.INTEGER: 'integer'>, params=None, points=0)}

# 검색

In [8]:
# vectorstore 이용해서 (유사도) 검색
query = "암울한 미래사회를 그린 소설을 추천해줘."
result = vectorstore.similarity_search(
    query = query, # 질문 -> str
    k=5     # 우선순위 높은 순서대로 해서 몇개 가져올지. TOP-K
)
result

[Document(metadata={'name': 'Brave New World', 'description': '유전자 조작과 세뇌로 계급이 철저히 나뉘어진 미래 사회가 배경이다. 인간은 자유 없이 쾌락과 안정을 주입받으며 살고, 비순응자는 배제된다. 디스토피아 속에서 인간 본성과 자유의 의미를 묻는 소설이다.', 'author': 'Aldous Huxley', 'year': 1932, '_id': '8e380646-6592-4908-b254-1e427f221dce', '_collection_name': 'sf_books'}, page_content='유전자 조작과 세뇌로 계급이 철저히 나뉘어진 미래 사회가 배경이다. 인간은 자유 없이 쾌락과 안정을 주입받으며 살고, 비순응자는 배제된다. 디스토피아 속에서 인간 본성과 자유의 의미를 묻는 소설이다.'),
 Document(metadata={'name': 'The Time Machine', 'description': '시간 여행자는 미래로 여행을 떠나 인류의 진화와 몰락을 목격하게 된다. 그는 멀리 떨어진 미래 사회에서 인간 문명이 퇴화한 모습을 보고 충격을 받는다. 이 이야기는 인간 문명과 과학기술의 진보가 가져올 결과를 철학적으로 탐구한다.', 'author': 'H.G. Wells', 'year': 1895, '_id': '31c12211-5bbd-497b-8492-69d9e8b544fb', '_collection_name': 'sf_books'}, page_content='시간 여행자는 미래로 여행을 떠나 인류의 진화와 몰락을 목격하게 된다. 그는 멀리 떨어진 미래 사회에서 인간 문명이 퇴화한 모습을 보고 충격을 받는다. 이 이야기는 인간 문명과 과학기술의 진보가 가져올 결과를 철학적으로 탐구한다.'),
 Document(metadata={'name': 'Foundation', 'description': "수학자 하리 셀던은 미래를 예측하는 '심리역사학'을 개발해 인류의 몰락을 막기 위한 계획을 세운다. 

In [28]:
# vectorstore 이용해서 (유사도) 검색
query = "암울한 미래사회를 그린 소설을 추천해줘."
result = vectorstore.similarity_search_with_score( # 유사도 점수까지 포함
    query=query, # 질문 -> str
    k=5,     # 우선순위 높은 순서대로 해서 몇개 가져올지. TOP-K
    score_threshold=0.3 # 지정한 점수보다 높은 것만 조회
)
result

[(Document(metadata={'name': 'Brave New World', 'description': '유전자 조작과 세뇌로 계급이 철저히 나뉘어진 미래 사회가 배경이다. 인간은 자유 없이 쾌락과 안정을 주입받으며 살고, 비순응자는 배제된다. 디스토피아 속에서 인간 본성과 자유의 의미를 묻는 소설이다.', 'author': 'Aldous Huxley', 'year': 1932, '_id': '8e380646-6592-4908-b254-1e427f221dce', '_collection_name': 'sf_books'}, page_content='유전자 조작과 세뇌로 계급이 철저히 나뉘어진 미래 사회가 배경이다. 인간은 자유 없이 쾌락과 안정을 주입받으며 살고, 비순응자는 배제된다. 디스토피아 속에서 인간 본성과 자유의 의미를 묻는 소설이다.'),
  0.3520673)]

In [29]:
#################
# Field Filtering
# Qdrant API 로 Filter 설정 직접해야 한다.
#################
from qdrant_client.models import Filter, FieldCondition, Range, MatchValue, MatchText

filter_cond = Filter(
    must=[
        FieldCondition(key="metadata.year", range=Range(lte=1000))
    ]
)
# filter를 먼저 하고, 유사도 체크
result = vectorstore.similarity_search(
    query=query,
    k=5,
    filter=filter_cond
)
result

[]

# Qdrant 세가지 방식

- **Dense Vector Search** (default): **의미 기반** 검색. Vector간 의미적 유사도록 검색
- **Sparse Vector Search**: **키워드 기반** 검색.
- **Hybrid Search**: 두가지 방법을 합쳐서 검색

## Dense Vector Search
- Default 방식
- `RetrievalMode.DENSE` 로 설정
- embedding 파라미터에 값이 제공되야 한다.

### Sparse Vector Search
- **희소 표현(sparse representation)을** 사용한 벡터 검색 방식.
- 이 기능은 특히 BM25, TF-IDF, Bag-of-Words 검색과 같은 전통적인 정보 검색(IR: Information Retrieval) 기법을 이용해 검색을 한다.
- 보통 이 방식을 단독으로 쓰지 않고 의미기반 검색을 보완하기 위해 사용한다. (Hybrid 검색)
- `pip install -qU fastembed`
  
#### 설정
- sparse vector search 만 할 경우
  - **collection 생성** 시 `sparse_vectors_config` 에 sparse vector 설정을 한다.
  - Langchain VectorStore 생성시 `sparse_embedding_model`을 설정하고 collection 생성시 지정한 설정을 `sparse_vector_name` 파라미터를 이용해 전달한다.

#### BM25 (Best Matching 25) 알고리즘

BM25는 검색 엔진이나 추천 시스템에서 **사용자가 입력한 검색어와 가장 관련성이 높은 문서**를 찾아 순위를 매기는 데 사용되는 알고리즘이다.   
통계 기반의 정보 검색 기법 중 하나로, 특히 문서 간의 관련성을 정량적으로 평가할 때 많이 사용된다.

BM25는 다음 세 가지 주요 요소를 바탕으로 점수를 계산한다:


##### 1. **단어의 출현 빈도 (Term Frequency, TF)**

- 문서 내 특정 단어가 얼마나 자주 등장하는지를 측정한다.
- 사용자가 검색한 단어가 어떤 문서에서 많이 등장할수록, 해당 문서는 검색어와 관련성이 높다고 판단한다.
- 하지만 너무 자주 등장하는 단어에 대해서는 **점수 증가를 둔화**시키는 방식(로그나 분수 기반의 함수)을 사용한다. 즉, 단어의 빈도 수가 많다고 무한정 점수가 올라가지는 않는다.

##### 2. **단어의 희소성 (Inverse Document Frequency, IDF)**

- 전체 문서(검색 대상 문서) 집합에서 특정 단어가 얼마나 희귀한지를 평가한다.
- 많은 문서에 동시에 등장하는 단어는 정보성이 낮기 때문에 낮은 점수를 부여하고, 드물게 등장하는 단어에는 높은 점수를 부여한다.

##### 3. **문서의 길이 보정 (Document Length Normalization)**

- 문서의 길이에 따라 점수를 보정한다.
- 같은 단어가 등장한 횟수가 같더라도, 짧은 문서에서는 그 단어의 중요도가 더 높다고 판단한다.
- 너무 긴 문서가 단지 길이 때문에 더 높은 점수를 받는 것을 방지하기 위해, 평균 문서 길이를 기준으로 보정한다.

### BM25 점수 계산 과정

1. **검색어 토큰화**
   - 사용자가 입력한 검색어 문장을 단어(토큰) 단위로 분리한다.
   - 예: "강아지 산책 방법" → \["강아지", "산책", "방법"\]
2. **각 문서에 대해 단어별 점수 계산**
   - 분리된 각 단어에 대해, 위 세 가지 요소를 기반으로 각 대상 문서의 점수를 계산한다:
3. **문서별 총점 산출**
   - 각 단어별 점수를 모두 더해 해당 문서의 총점을 계산한다.
4. **문서 순위화**
   - 점수가 높은 문서일수록 검색어와 관련성이 높다고 판단하여, 그 순서대로 결과를 정렬한다.



### BM25의 한계

- **의미 파악 불가**: BM25는 단어의 단순 일치 여부만 평가하며, 단어의 의미나 문맥은 고려하지 않는다. 예를 들어 "사과"가 "과일"인지 "용서"인지 구분하지 못한다.
- **동의어 처리 어려움**: "자동차"와 "승용차"처럼 다른 단어지만 유사한 의미를 가진 경우, BM25는 이들을 동일하게 인식하지 못한다.
- **자연어 이해 부족**: 문장의 구조나 어순, 구문 정보는 활용하지 않는다.
- **언어별 특성**- 한국어처럼 교착어에서는 형태소 분석이 중요한데 이에 대한 고려가 부족
- **구문 관계**: 단어 간의 거리나 위치 정보를 고려하지 않음

BM25는 간단하면서도 강력한 정보 검색 알고리즘으로, **키워드 기반 검색**에 매우 효과적이다. 다만 단어의 **의미나 문맥을 이해하지 못한다는 점**에서 **의미 기반 검색 모델**이 보완적으로 사용되기도 한다.


In [30]:
!uv pip install fastembed

Resolved 29 packages in 193ms
Prepared 5 packages in 88ms
Installed 5 packages in 37ms
 + fastembed==0.7.4
 + loguru==0.7.3
 + mmh3==5.2.0
 + py-rust-stemmers==0.1.5
 + win32-setctime==1.2.0


In [4]:
from langchain_qdrant import FastEmbedSparse, QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, SparseVectorParams, VectorParams, SparseIndexParams
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

# huggingface login
import os
from huggingface_hub import login

load_dotenv()
login(os.getenv('HUGGINGFACE_API_KEY'))

In [5]:
COLLECTION_NAME = "sf_book_sparse"
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

client = QdrantClient(host="localhost", port=6333)

# 삭제
if client.collection_exists(COLLECTION_NAME):
    client.delete_collection(COLLECTION_NAME)

# Collection 생성 -> Sparse Vector 저장소.
client.create_collection(
    collection_name=COLLECTION_NAME,
    # Sparse vector 저장소 설정.
    sparse_vectors_config= { # Dict - key:저장소이름, value: 설정
        "sparse":SparseVectorParams(index=SparseIndexParams(on_disk=False)) # index sparse vector를 메모리(False)/디스크(True) 어디에 저장할지.
    }
)

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

arabic.txt: 0.00B [00:00, ?B/s]

finnish.txt: 0.00B [00:00, ?B/s]

c:\Users\Playdata\Documents\SKN21_2\SKN21\10_langchain\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\AppData\Local\Temp\fastembed_cache\models--Qdrant--bm25. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


english.txt:   0%|          | 0.00/936 [00:00<?, ?B/s]

...

romanian.txt: 0.00B [00:00, ?B/s]

portuguese.txt: 0.00B [00:00, ?B/s]

swedish.txt:   0%|          | 0.00/559 [00:00<?, ?B/s]

turkish.txt:   0%|          | 0.00/260 [00:00<?, ?B/s]

True

In [ ]:
# VectorStore
sparse_vectorstore = QdrantVectorStore(
    client=client,
    collection_name=COLLECTION_NAME,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.SPARSE, ## 문서 검색 방식 - Default: Dense Vector 방식, HYBRID
    sparse_vector_name="sparse" # Sparse Vector 저장소 이름. sparse_vectors_config에서 설정한 key값.
)
# 데이터 upsert
sparse_vectorstore.add_documents(documents)

['dda79020-87e1-4027-8d92-f056ec4fca67',
 '4a780efc-434f-4146-85c8-ff7b779a572d',
 '08717161-3816-470a-9f0b-8747de84fde7',
 '09f26a5d-9644-4a5a-ba91-91bd23e2f6fe',
 'a9f19967-b68b-4a82-a684-26d5e7c07930',
 '15a5fb65-4451-4bd5-93af-b4833a431600',
 'a91425eb-d889-4dce-bde7-f5049c912881',
 '5e70ef06-702d-4c47-90f9-1ea53a9b946f',
 'ada5e173-ab4a-451a-9cb4-726a4509144d',
 '0213c96f-dced-4475-996a-76cf4915a0f7',
 '2f0f938c-c0b4-484b-b6f8-b16d0ee69065',
 'a139a25a-aae8-4f00-bdd5-8d2fd4823690',
 '2bd21946-5e0a-448b-a721-b814de0072ca']

In [9]:
query = "암울한 미래세계에 대한 소설을 추천해줘."
query = "우주 여행과 외계 생명체에 대한 소설"
result = sparse_vectorstore.similarity_search_with_score(
    query, k=5
)
result

[(Document(metadata={'name': "The Hitchhiker's Guide to the Galaxy", 'description': '지구 멸망 직후 우주로 탈출한 주인공이 외계 친구와 함께 기상천외한 모험을 시작한다. 우주의 황당한 존재들과 철학적 농담들이 넘치는 코믹 SF다. 우주에 대한 진지한 질문을 유쾌하게 풀어낸 작품이다.', 'author': 'Douglas Adams', 'year': 1979, '_id': '09f26a5d-9644-4a5a-ba91-91bd23e2f6fe', '_collection_name': 'sf_book_sparse'}, page_content='지구 멸망 직후 우주로 탈출한 주인공이 외계 친구와 함께 기상천외한 모험을 시작한다. 우주의 황당한 존재들과 철학적 농담들이 넘치는 코믹 SF다. 우주에 대한 진지한 질문을 유쾌하게 풀어낸 작품이다.'),
  3.1542985),
 (Document(metadata={'name': 'The Left Hand of Darkness', 'description': '외교관 제너리 아이가 젠더가 유동적인 외계 종족이 사는 행성에 파견된다. 그는 그들과의 문화적 충돌과 이해를 통해 진정한 인간성과 연대를 배워간다. 젠더와 정체성, 타자성에 대한 철학적 탐구가 돋보이는 작품이다.', 'author': 'Ursula K. Le Guin', 'year': 1969, '_id': 'a139a25a-aae8-4f00-bdd5-8d2fd4823690', '_collection_name': 'sf_book_sparse'}, page_content='외교관 제너리 아이가 젠더가 유동적인 외계 종족이 사는 행성에 파견된다. 그는 그들과의 문화적 충돌과 이해를 통해 진정한 인간성과 연대를 배워간다. 젠더와 정체성, 타자성에 대한 철학적 탐구가 돋보이는 작품이다.'),
  3.1463687),
 (Document(metadata={'name': 'The War of the Worlds', '

### Hybrid
- Hybrid Search는 Dense Vector Search와 Sparse Vector Search를 결합하여, **의미 기반 검색(semantic search)**과 **정확한 키워드 검색(lexical search)**의 장점을 모두 활용하는 검색 방식이다.
- RAG(Retrieval-Augmented Generation) 시스템이나 검색 기반 질문응답 시스템에서 정확도와 재현율을 동시에 향상시키는 데 매우 유용하다.

> - Dense Vector: 문장의 의미를 파악하는 임베딩 기반 검색
> - Sparse Vector: 키워드 기반의 전통 정보 검색(BM25, TF-IDF 등)

- Dense vector만 사용하면 문맥적 의미는 이해하지만 키워드가 누락된 문서가 나올 수있다.
- Sparse vector만 사용하면 정확한 단어, 키워드는 찾지만 의미 파악이 부족할 수있다.
- 위 두가지 방식의 단점을 보완한 것이 Hybrid 방식으로 **정확한 키워드와 의미적으로 유사한 문서**를 함께 찾는다.
- 하이브리드 방식을 적용하면
  - BM25로 1차 후보 검색
  - Dense Embedding으로 의미 기반 재정렬

In [1]:
from langchain_qdrant import FastEmbedSparse, QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, SparseVectorParams, VectorParams, SparseIndexParams
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

# huggingface login
import os
from huggingface_hub import login

load_dotenv()
login(os.getenv('HUGGINGFACE_API_KEY'))

In [2]:
COLLECTION_NAME = "sf_book_hybrid"
# BM25 알고리즘으로 입력 문서를 vector화 하는 모델
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")
# 의미기반으로 입력 문서를 vector화하는 embedding 모델
dense_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
VECTOR_SIZE = 1536

client = QdrantClient(host="localhost", port=6333)

# 삭제
if client.collection_exists(COLLECTION_NAME):
    client.delete_collection(COLLECTION_NAME)

# Collection 생성 -> Sparse Vector 저장소.
client.create_collection(
    collection_name=COLLECTION_NAME,
    # Sparse vector 저장소 설정.
    sparse_vectors_config= { # Dict - key:저장소이름, value: 설정
        "sparse":SparseVectorParams(index=SparseIndexParams(on_disk=False)) # index sparse vector를 메모리(False)/디스크(True) 어디에 저장할지.
    }, # Sparse Vector 저장소 설정
    vectors_config={
        "dense":VectorParams(size=VECTOR_SIZE, distance=Distance.COSINE)
    } # Dense Vector 저장소 설정
)

True

In [7]:
# vector store
hybrid_vectorstore = QdrantVectorStore(
    client=client,
    collection_name=COLLECTION_NAME, # DENSE, SPARSE 두개 저장소
    
    embedding=dense_embeddings,
    sparse_embedding=sparse_embeddings,

    retrieval_mode=RetrievalMode.HYBRID,

    vector_name="dense",
    sparse_vector_name="sparse"
)

hybrid_vectorstore.add_documents(documents)

['18bbabf7-3176-484e-b9e1-29af968df565',
 '774c53ac-fa42-46f9-985f-38f22ad77a41',
 '60409d7c-ef94-4890-94c7-fa5a99f4a016',
 'd873b0b3-7b21-4d3f-9ba0-1fb904fc1fc0',
 '4b08e06d-227f-4517-a043-2fea91cd60ef',
 'c9f97405-7adb-4b0d-bc8b-50282e853173',
 '4744aeaa-d408-4d44-8b98-fb7775b48368',
 '7af01638-9b10-4f2d-96cc-039b0eb35d71',
 'c6890e51-c006-4d4b-b4d9-22272c9613df',
 'd5ceda54-b741-44ab-8b00-c06e7a314ce8',
 '53954a81-7ca4-49fc-af3f-dd963e4801ee',
 'cb0f6420-6bbe-4057-aa65-401d9d91b05b',
 '34ce7b54-bf6e-487e-8615-a3a686cf4ab9']

In [8]:
result, next_id = client.scroll(
    collection_name=COLLECTION_NAME,
    limit=1,
    with_vectors=True, # 조회시 Vector도 조회 (디폴트 False)
    with_payload=False, # 조회시 payload 조회여부(디폴트 True)
)

In [9]:
d = result[0].vector
d.keys()

dict_keys(['sparse', 'dense'])

In [16]:
print(len(d['dense']))
d['sparse']
# indices:단어 index, values:단어별 점수
print(len(d['sparse'].indices))
print(len(d['sparse'].values))

1536
31
31


In [ ]:
query = "디스토피아적 세계관을 가진 소설을 추천해줘."
# Dense vector와의 유사도 점수(의미적 유사도), Sparse Vector와의 유사도 점수(키워드 기반)를 조합(합성) 해서 최종 rank를 계산
# 이 result 코드를 해주는 Runnable이 있는데 그게 Retriever임. 
result = hybrid_vectorstore.similarity_search_with_score(
    query=query,
    k=5
)
result

[(Document(metadata={'name': 'The Hunger Games', 'description': '디스토피아 사회에서 어린이들은 생존 게임에 강제로 참가해야 한다. 주인공 캣니스는 잔혹한 게임 속에서 인간성과 저항의 상징이 되어간다. 권력과 억압, 생존 본능의 이야기가 펼쳐진다.', 'author': 'Suzanne Collins', 'year': 2008, '_id': 'd5ceda54-b741-44ab-8b00-c06e7a314ce8', '_collection_name': 'sf_book_hybrid'}, page_content='디스토피아 사회에서 어린이들은 생존 게임에 강제로 참가해야 한다. 주인공 캣니스는 잔혹한 게임 속에서 인간성과 저항의 상징이 되어간다. 권력과 억압, 생존 본능의 이야기가 펼쳐진다.'),
  0.5),
 (Document(metadata={'name': 'The Left Hand of Darkness', 'description': '외교관 제너리 아이가 젠더가 유동적인 외계 종족이 사는 행성에 파견된다. 그는 그들과의 문화적 충돌과 이해를 통해 진정한 인간성과 연대를 배워간다. 젠더와 정체성, 타자성에 대한 철학적 탐구가 돋보이는 작품이다.', 'author': 'Ursula K. Le Guin', 'year': 1969, '_id': 'cb0f6420-6bbe-4057-aa65-401d9d91b05b', '_collection_name': 'sf_book_hybrid'}, page_content='외교관 제너리 아이가 젠더가 유동적인 외계 종족이 사는 행성에 파견된다. 그는 그들과의 문화적 충돌과 이해를 통해 진정한 인간성과 연대를 배워간다. 젠더와 정체성, 타자성에 대한 철학적 탐구가 돋보이는 작품이다.'),
  0.33333334),
 (Document(metadata={'name': 'Brave New World', 'description': '유전자 조작과 세뇌로 계급이 철저히 나뉘어진 미래 사회가 배경